# Versions
1.0  
-> Primeiro teste  
1.1  
-> Trocado copy_rates_range para copy_rates_from_pos por conta da perda de dados  
-> Passando funções para robot_functions.py  
-> Primeira implementação de uma GUI

# O indice 0 é o mais antigo, o ultimo é o mais novo
# Jogar funções em um modulo
# Entender o resultado do account_info
# Fazer uma gui e com o schedule dar quebra no while true da gui para ir ao while true geral e pegar dados?

In [1]:
import MetaTrader5 as mt5
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import numpy as np
import robot_functions as func
import schedule as sc
from time import sleep
from IPython.display import clear_output
from PySimpleGUI import PySimpleGUI as sg

plt.style.use('dark_background')
plt.rcParams["figure.figsize"] = (30,10)

In [2]:
# Constantes

pairs = [
        ['EURCHF','EURGBP','EURJPY','EURNZD','EURUSD','EURAUD','EURCAD'],
        ['EURGBP','GBPAUD','GBPCHF','GBPJPY','GBPCAD','GBPUSD','GBPNZD'],
        ['GBPUSD','USDCHF','USDJPY','AUDUSD','NZDUSD','USDCAD','EURUSD'],
        ['AUDJPY','CADJPY','CHFJPY','EURJPY','USDJPY','GBPJPY','NZDJPY'],
        ['AUDCHF','CADCHF','CHFJPY','USDCHF','EURCHF','GBPCHF','NZDCHF'],
        ['AUDNZD','EURNZD','GBPNZD','NZDUSD','NZDCAD','NZDCHF','NZDJPY'],
        ['AUDCAD','AUDCHF','AUDJPY','AUDUSD','AUDNZD','EURAUD','GBPAUD'],
        ['AUDCAD','CADCHF','CADJPY','USDCAD','EURCAD','GBPCAD','NZDCAD']
        ]

all_pairs = ['eur','gbp','usd','jpy','chf','nzd','aud','cad']

risk = 0.01

data = pd.DataFrame()
test_data = pd.DataFrame()

ea_magic_number = 1010

order_request_buy_eurusd, order_request_sell_eurusd = 0, 0
flag_buy_eurusd, flag_sell_eurusd = True, True

best_time = 7

eur_best_time = 7
usd_best_time = 7
gbp_best_time = 7
jpy_best_time = 7
chf_best_time = 7
nzd_best_time = 7
aud_best_time = 7

In [8]:
func.login_mt5(login=40305620,password='x8dcteyv',server='MetaQuotes-Demo')


sg.theme('SystemDefault')

layout = [
    [sg.Text("",size=(20,1),key='-EUR0-',justification='left')],
    [sg.Text("",size=(20,1),key='-USD0-',justification='left')],
    [sg.Text("",size=(20,1),key='-GBP0-',justification='left')],
    [sg.Text("",size=(20,1),key='-JPY0-',justification='left')],
    [sg.Text("",size=(20,1),key='-CHF0-',justification='left')],
    [sg.Text("",size=(20,1),key='-NZD0-',justification='left')],
    [sg.Text("",size=(20,1),key='-AUD0-',justification='left')],
    [sg.Text("",size=(20,1),key='-CAD0-',justification='left')],
    [sg.Button('Exit')]
]

window = sg.Window('Robot V1.1',layout,auto_size_text=True)

while True:
    
    # Get Data
    for i in range(len(pairs)):
        data[all_pairs[i]] = func.diff_data(list_symbol = pairs[i], start = 0, end = 10)
    data.dropna(inplace=True)

    leverage, margin_free, balance, profit = func.account_information()
    
    event, _ = window.read(timeout=1)
    if event == sg.WIN_CLOSED or event == 'Exit':
        exit_flag = True
        window.close()
        break

    window['-EUR0-'].update(f"EUR - C0: {data['eur'].iloc[-1]} | C-1: {data['eur'].iloc[-2]}")
    window['-USD0-'].update(f"USD - C0: {data['usd'].iloc[-1]} | C-1: {data['usd'].iloc[-2]}")
    window['-GBP0-'].update(f"GBP - C0: {data['gbp'].iloc[-1]} | C-1: {data['gbp'].iloc[-2]}")
    window['-JPY0-'].update(f"JPY - C0: {data['jpy'].iloc[-1]} | C-1: {data['jpy'].iloc[-2]}")
    window['-CHF0-'].update(f"CHF - C0: {data['chf'].iloc[-1]} | C-1: {data['chf'].iloc[-2]}")
    window['-NZD0-'].update(f"NZD - C0: {data['nzd'].iloc[-1]} | C-1: {data['nzd'].iloc[-2]}")
    window['-AUD0-'].update(f"AUD - C0: {data['aud'].iloc[-1]} | C-1: {data['aud'].iloc[-2]}")
    window['-CAD0-'].update(f"CAD - C0: {data['cad'].iloc[-1]} | C-1: {data['cad'].iloc[-2]}")

            #EUR
    # EURUSD
    if data['eur'].iloc[-1] < -best_time and data['usd'].iloc[-1] > best_time and data['eur'].iloc[-2] > -best_time and data['usd'].iloc[-2] < best_time and flag_buy_eurusd:
        if not flag_sell_eurusd:
            func.close_trader('buy',order_request_sell_eurusd)
            order_request_buy_eurusd = func.open_trade(action='buy', symbol='EURUSD', balance=margin_free, risk=risk, leverage=leverage)
            flag_sell_eurusd = True
            flag_buy_eurusd = False
        else:
            order_request_buy_eurusd = func.open_trade(action='buy', symbol='EURUSD', balance=margin_free, risk=risk, leverage=leverage)
            flag_buy_eurusd = False
    elif data['eur'].iloc[-1] > best_time and data['usd'].iloc[-1] < -best_time and data['eur'].iloc[-2] < best_time and data['usd'].iloc[-2] > -best_time and flag_sell_eurusd:
        if not flag_buy_eurusd:
            func.close_trader('sell',order_request_buy_eurusd)
            order_request_sell_eurusd = func.open_trade(action='sell', symbol='EURUSD', balance=margin_free, risk=risk, leverage=leverage)
            flag_buy_eurusd = True
            flag_sell_eurusd = False
        else:
            order_request_sell_eurusd = func.open_trade(action='sell', symbol='EURUSD', balance=margin_free, risk=risk, leverage=leverage)
            flag_sell_eurusd = False
            
    sleep(0.5)

mt5.shutdown()

True

In [ ]:
data

In [ ]:
def multi_backtest(initial,final):
    
eur_best_time = 7
usd_best_time = 7
gbp_best_time = 7
jpy_best_time = 7
chf_best_time = 7
nzd_best_time = 7
aud_best_time = 7
    
    # EUR
    flag_buy_eurusd, flag_sell_eurusd = True, True
    flag_buy_eurgbp, flag_sell_eurgbp = True, True
    flag_buy_eurchf, flag_sell_eurchf = True, True
    flag_buy_eurnzd, flag_sell_eurnzd = True, True
    flag_buy_euraud, flag_sell_euraud = True, True
    flag_buy_eurcad, flag_sell_eurcad = True, True
    flag_buy_eurjpy, flag_sell_eurjpy = True, True

    # USD
    flag_buy_gbpusd, flag_sell_gbpusd = True, True
    flag_buy_usdchf, flag_sell_usdchf = True, True
    flag_buy_usdjpy, flag_sell_usdjpy = True, True
    flag_buy_audusd, flag_sell_audusd = True, True
    flag_buy_nzdusd, flag_sell_nzdusd = True, True
    flag_buy_usdcad, flag_sell_usdcad = True, True

    # GBP
    flag_buy_gbpaud, flag_sell_gbpaud = True, True
    flag_buy_gbpchf, flag_sell_gbpchf = True, True
    flag_buy_gbpjpy, flag_sell_gbpjpy = True, True
    flag_buy_gbpcad, flag_sell_gbpcad = True, True
    flag_buy_gbpnzd, flag_sell_gbpnzd = True, True

    # JPY
    flag_buy_audjpy, flag_sell_audjpy = True, True
    flag_buy_cadjpy, flag_sell_cadjpy = True, True
    flag_buy_chfjpy, flag_sell_chfjpy = True, True
    flag_buy_nzdjpy, flag_sell_nzdjpy = True, True

    # CHF
    flag_buy_audchf, flag_sell_audchf = True, True
    flag_buy_cadchf, flag_sell_cadchf = True, True
    flag_buy_nzdchf, flag_sell_nzdchf = True, True

    # NZD
    flag_buy_audnzd, flag_sell_audnzd = True, True
    flag_buy_nzdcad, flag_sell_nzdcad = True, True

    # AUD
    flag_buy_audcad, flag_sell_audcad = True, True


    #EURGBP
    elif data['eur_low'].iloc[i] < -eur_best_time and data['gbp_high'].iloc[i] > eur_best_time and flag_buy_eurgbp:
        if not flag_sell_eurgbp:
            balance_backtest = account_balance(balance_backtest,risk,leverage,price_sold_eurgbp,data['EURGBP_Low'].iloc[i])
            list_backtest.append(balance_backtest)
            price_bought_eurgbp = data['EURGBP_Low'].iloc[i]
            price_sold_eurgbp = 0
            flag_sell_eurgbp = True
            flag_buy_eurgbp = False
        else:
            price_bought_eurgbp = data['EURGBP_Low'].iloc[i]
            flag_buy_eurgbp = False
    elif data['eur_high'].iloc[i] > eur_best_time and data['gbp_low'].iloc[i] < -eur_best_time and flag_sell_eurgbp:
        if not flag_buy_eurgbp:
            balance_backtest = account_balance(balance_backtest,risk,leverage,data['EURGBP_High'].iloc[i],price_bought_eurgbp)
            list_backtest.append(balance_backtest)
            price_sold_eurgbp = data['EURGBP_High'].iloc[i]
            price_bought_eurgbp = 0
            flag_buy_eurgbp = True
            flag_sell_eurgbp = False
        else:
            price_sold_eurgbp = data['EURGBP_High'].iloc[i]
            flag_sell_eurgbp = False
    #EURCHF
    elif data['eur_low'].iloc[i] < -eur_best_time and data['chf_high'].iloc[i] > eur_best_time and flag_buy_eurchf:
        if not flag_sell_eurchf:
            balance_backtest = account_balance(balance_backtest,risk,leverage,price_sold_eurchf,data['EURCHF_Low'].iloc[i])
            list_backtest.append(balance_backtest)
            price_bought_eurchf = data['EURCHF_Low'].iloc[i]
            price_sold_eurchf = 0
            flag_sell_eurchf = True
            flag_buy_eurchf = False
        else:
            price_bought_eurchf = data['EURCHF_Low'].iloc[i]
            flag_buy_eurchf = False
    elif data['eur_high'].iloc[i] > eur_best_time and data['chf_low'].iloc[i] < -eur_best_time and flag_sell_eurchf:
        if not flag_buy_eurchf:
            balance_backtest = account_balance(balance_backtest,risk,leverage,data['EURCHF_High'].iloc[i],price_bought_eurchf)
            list_backtest.append(balance_backtest)
            price_sold_eurchf = data['EURCHF_High'].iloc[i]
            price_bought_eurchf = 0
            flag_buy_eurchf = True
            flag_sell_eurchf = False
        else:
            price_sold_eurchf = data['EURCHF_High'].iloc[i]
            flag_sell_eurchf = False
    #EURNZD
    elif data['eur_low'].iloc[i] < -eur_best_time and data['nzd_high'].iloc[i] > eur_best_time and flag_buy_eurnzd:
        if not flag_sell_eurnzd:
            balance_backtest = account_balance(balance_backtest,risk,leverage,price_sold_eurnzd,data['EURNZD_Low'].iloc[i])
            list_backtest.append(balance_backtest)
            price_bought_eurnzd = data['EURNZD_Low'].iloc[i]
            price_sold_eurnzd = 0
            flag_sell_eurnzd = True
            flag_buy_eurnzd = False
        else:
            price_bought_eurnzd = data['EURNZD_Low'].iloc[i]
            flag_buy_eurnzd = False
    elif data['eur_high'].iloc[i] > eur_best_time and data['nzd_low'].iloc[i] < -eur_best_time and flag_sell_eurnzd:
        if not flag_buy_eurnzd:
            balance_backtest = account_balance(balance_backtest,risk,leverage,data['EURNZD_High'].iloc[i],price_bought_eurnzd)
            list_backtest.append(balance_backtest)
            price_sold_eurnzd = data['EURNZD_High'].iloc[i]
            price_bought_eurnzd = 0
            flag_buy_eurnzd = True
            flag_sell_eurnzd = False
        else:
            price_sold_eurnzd = data['EURNZD_High'].iloc[i]
            flag_sell_eurnzd = False
    #EURAUD
    elif data['eur_low'].iloc[i] < -eur_best_time and data['aud_high'].iloc[i] > eur_best_time and flag_buy_euraud:
        if not flag_sell_euraud:
            balance_backtest = account_balance(balance_backtest,risk,leverage,price_sold_euraud,data['EURAUD_Low'].iloc[i])
            list_backtest.append(balance_backtest)
            price_bought_euraud = data['EURAUD_Low'].iloc[i]
            price_sold_euraud = 0
            flag_sell_euraud = True
            flag_buy_euraud = False
        else:
            price_bought_euraud = data['EURAUD_Low'].iloc[i]
            flag_buy_euraud = False
    elif data['eur_high'].iloc[i] > eur_best_time and data['aud_low'].iloc[i] < -eur_best_time and flag_sell_euraud:
        if not flag_buy_euraud:
            balance_backtest = account_balance(balance_backtest,risk,leverage,data['EURAUD_High'].iloc[i],price_bought_euraud)
            list_backtest.append(balance_backtest)
            price_sold_euraud = data['EURAUD_High'].iloc[i]
            price_bought_euraud = 0
            flag_buy_euraud = True
            flag_sell_euraud = False
        else:
            price_sold_euraud = data['EURAUD_High'].iloc[i]
            flag_sell_euraud = False
    #EURCAD
    elif data['eur_low'].iloc[i] < -eur_best_time and data['cad_high'].iloc[i] > eur_best_time and flag_buy_eurcad:
        if not flag_sell_eurcad:
            balance_backtest = account_balance(balance_backtest,risk,leverage,price_sold_eurcad,data['EURCAD_Low'].iloc[i])
            list_backtest.append(balance_backtest)
            price_bought_eurcad = data['EURCAD_Low'].iloc[i]
            price_sold_eurcad = 0
            flag_sell_eurcad = True
            flag_buy_eurcad = False
        else:
            price_bought_eurcad = data['EURCAD_Low'].iloc[i]
            flag_buy_eurcad = False
    elif data['eur_high'].iloc[i] > eur_best_time and data['cad_low'].iloc[i] < -eur_best_time and flag_sell_eurcad:
        if not flag_buy_eurcad:
            balance_backtest = account_balance(balance_backtest,risk,leverage,data['EURCAD_High'].iloc[i],price_bought_eurcad)
            list_backtest.append(balance_backtest)
            price_sold_eurcad = data['EURCAD_High'].iloc[i]
            price_bought_eurcad = 0
            flag_buy_eurcad = True
            flag_sell_eurcad = False
        else:
            price_sold_eurcad = data['EURCAD_High'].iloc[i]
            flag_sell_eurcad = False
    #EURJPY
    elif data['eur_low'].iloc[i] < -eur_best_time and data['jpy_high'].iloc[i] > eur_best_time and flag_buy_eurjpy:
        if not flag_sell_eurjpy:
            balance_backtest = account_balance(balance_backtest,risk,leverage,price_sold_eurjpy,data['EURJPY_Low'].iloc[i],jpy=True)
            list_backtest.append(balance_backtest)
            price_bought_eurjpy = data['EURJPY_Low'].iloc[i]
            price_sold_eurjpy = 0
            flag_sell_eurjpy = True
            flag_buy_eurjpy = False
        else:
            price_bought_eurjpy = data['EURJPY_Low'].iloc[i]
            flag_buy_eurjpy = False
    elif data['eur_high'].iloc[i] > eur_best_time and data['jpy_low'].iloc[i] < -eur_best_time and flag_sell_eurjpy:
        if not flag_buy_eurjpy:
            balance_backtest = account_balance(balance_backtest,risk,leverage,data['EURJPY_High'].iloc[i],price_bought_eurjpy,jpy=True)
            list_backtest.append(balance_backtest)
            price_sold_eurjpy = data['EURJPY_High'].iloc[i]
            price_bought_eurjpy = 0
            flag_buy_eurjpy = True
            flag_sell_eurjpy = False
        else:
            price_sold_eurjpy = data['EURJPY_High'].iloc[i]
            flag_sell_eurjpy = False
            # USD
   # GBPUSD
    elif data['gbp_low'].iloc[i] < -usd_best_time and data['usd_high'].iloc[i] > usd_best_time and flag_buy_gbpusd:
        if not flag_sell_gbpusd:
            balance_backtest = account_balance(balance_backtest,risk,leverage,price_sold_gbpusd,data['GBPUSD_Low'].iloc[i])
            list_backtest.append(balance_backtest)
            price_bought_gbpusd = data['GBPUSD_Low'].iloc[i]
            price_sold_gbpusd = 0
            flag_sell_gbpusd = True
            flag_buy_gbpusd = False
        else:
            price_bought_gbpusd = data['GBPUSD_Low'].iloc[i]
            flag_buy_gbpusd = False
    elif data['gbp_high'].iloc[i] > usd_best_time and data['usd_low'].iloc[i] < -usd_best_time and flag_sell_gbpusd:
        if not flag_buy_gbpusd:
            balance_backtest = account_balance(balance_backtest,risk,leverage,data['GBPUSD_High'].iloc[i],price_bought_gbpusd)
            list_backtest.append(balance_backtest)
            price_sold_gbpusd = data['GBPUSD_High'].iloc[i]
            price_bought_gbpusd = 0
            flag_buy_gbpusd = True
            flag_sell_gbpusd = False
        else:
            price_sold_gbpusd = data['GBPUSD_High'].iloc[i]
            flag_sell_gbpusd = False
    # USDCHF
    elif data['usd_low'].iloc[i] < -usd_best_time and data['chf_high'].iloc[i] > usd_best_time and flag_buy_usdchf:
        if not flag_sell_usdchf:
            balance_backtest = account_balance(balance_backtest,risk,leverage,price_sold_usdchf,data['USDCHF_Low'].iloc[i])
            list_backtest.append(balance_backtest)
            price_bought_usdchf = data['USDCHF_Low'].iloc[i]
            price_sold_usdchf = 0
            flag_sell_usdchf = True
            flag_buy_usdchf = False
        else:
            price_bought_usdchf = data['USDCHF_Low'].iloc[i]
            flag_buy_usdchf = False
    elif data['usd_high'].iloc[i] > usd_best_time and data['chf_low'].iloc[i] < -usd_best_time and flag_sell_usdchf:
        if not flag_buy_usdchf:
            balance_backtest = account_balance(balance_backtest,risk,leverage,data['USDCHF_High'].iloc[i],price_bought_usdchf)
            list_backtest.append(balance_backtest)
            price_sold_usdchf = data['USDCHF_High'].iloc[i]
            price_bought_usdchf = 0
            flag_buy_usdchf = True
            flag_sell_usdchf = False
        else:
            price_sold_usdchf = data['USDCHF_High'].iloc[i]
            flag_sell_usdchf = False
    # USDJPY
    elif data['usd_low'].iloc[i] < -usd_best_time and data['jpy_high'].iloc[i] > usd_best_time and flag_buy_usdjpy:
        if not flag_sell_usdjpy:
            balance_backtest = account_balance(balance_backtest,risk,leverage,price_sold_usdjpy,data['USDJPY_Low'].iloc[i],jpy=True)
            list_backtest.append(balance_backtest)
            price_bought_usdjpy = data['USDJPY_Low'].iloc[i]
            price_sold_usdjpy = 0
            flag_sell_usdjpy = True
            flag_buy_usdjpy = False
        else:
            price_bought_usdjpy = data['USDJPY_Low'].iloc[i]
            flag_buy_usdjpy = False
    elif data['usd_high'].iloc[i] > usd_best_time and data['jpy_low'].iloc[i] < -usd_best_time and flag_sell_usdjpy:
        if not flag_buy_usdjpy:
            balance_backtest = account_balance(balance_backtest,risk,leverage,data['USDJPY_High'].iloc[i],price_bought_usdjpy,jpy=True)
            list_backtest.append(balance_backtest)
            price_sold_usdjpy = data['USDJPY_High'].iloc[i]
            price_bought_usdjpy = 0
            flag_buy_usdjpy = True
            flag_sell_usdjpy = False
        else:
            price_sold_usdjpy = data['USDJPY_High'].iloc[i]
            flag_sell_usdjpy = False
    # AUDUSD
    elif data['aud_low'].iloc[i] < -usd_best_time and data['usd_high'].iloc[i] > usd_best_time and flag_buy_audusd:
        if not flag_sell_audusd:
            balance_backtest = account_balance(balance_backtest,risk,leverage,price_sold_audusd,data['AUDUSD_Low'].iloc[i])
            list_backtest.append(balance_backtest)
            price_bought_audusd = data['AUDUSD_Low'].iloc[i]
            price_sold_audusd = 0
            flag_sell_audusd = True
            flag_buy_audusd = False
        else:
            price_bought_audusd = data['AUDUSD_Low'].iloc[i]
            flag_buy_audusd = False
    elif data['aud_high'].iloc[i] > usd_best_time and data['usd_low'].iloc[i] < -usd_best_time and flag_sell_audusd:
        if not flag_buy_audusd:
            balance_backtest = account_balance(balance_backtest,risk,leverage,data['AUDUSD_High'].iloc[i],price_bought_audusd)
            list_backtest.append(balance_backtest)
            price_sold_audusd = data['AUDUSD_High'].iloc[i]
            price_bought_audusd = 0
            flag_buy_audusd = True
            flag_sell_audusd = False
        else:
            price_sold_audusd = data['AUDUSD_High'].iloc[i]
            flag_sell_audusd = False
    # NZDUSD
    elif data['nzd_low'].iloc[i] < -usd_best_time and data['usd_high'].iloc[i] > usd_best_time and flag_buy_nzdusd:
        if not flag_sell_nzdusd:
            balance_backtest = account_balance(balance_backtest,risk,leverage,price_sold_nzdusd,data['NZDUSD_Low'].iloc[i])
            list_backtest.append(balance_backtest)
            price_bought_nzdusd = data['NZDUSD_Low'].iloc[i]
            price_sold_nzdusd = 0
            flag_sell_nzdusd = True
            flag_buy_nzdusd = False
        else:
            price_bought_nzdusd = data['NZDUSD_Low'].iloc[i]
            flag_buy_nzdusd = False
    elif data['nzd_high'].iloc[i] > usd_best_time and data['usd_low'].iloc[i] < -usd_best_time and flag_sell_nzdusd:
        if not flag_buy_nzdusd:
            balance_backtest = account_balance(balance_backtest,risk,leverage,data['NZDUSD_High'].iloc[i],price_bought_nzdusd)
            list_backtest.append(balance_backtest)
            price_sold_nzdusd = data['NZDUSD_High'].iloc[i]
            price_bought_nzdusd = 0
            flag_buy_nzdusd = True
            flag_sell_nzdusd = False
        else:
            price_sold_nzdusd = data['NZDUSD_High'].iloc[i]
            flag_sell_nzdusd = False
    # USDCAD
    elif data['usd_low'].iloc[i] < -usd_best_time and data['cad_high'].iloc[i] > usd_best_time and flag_buy_usdcad:
        if not flag_sell_usdcad:
            balance_backtest = account_balance(balance_backtest,risk,leverage,price_sold_usdcad,data['USDCAD_Low'].iloc[i])
            list_backtest.append(balance_backtest)
            price_bought_usdcad = data['USDCAD_Low'].iloc[i]
            price_sold_usdcad = 0
            flag_sell_usdcad = True
            flag_buy_usdcad = False
        else:
            price_bought_usdcad = data['USDCAD_Low'].iloc[i]
            flag_buy_usdcad = False
    elif data['usd_high'].iloc[i] > usd_best_time and data['cad_low'].iloc[i] < -usd_best_time and flag_sell_usdcad:
        if not flag_buy_usdcad:
            balance_backtest = account_balance(balance_backtest,risk,leverage,data['USDCAD_High'].iloc[i],price_bought_usdcad)
            list_backtest.append(balance_backtest)
            price_sold_usdcad = data['USDCAD_High'].iloc[i]
            price_bought_usdcad = 0
            flag_buy_usdcad = True
            flag_sell_usdcad = False
        else:
            price_sold_usdcad = data['USDCAD_High'].iloc[i]
            flag_sell_usdcad = False
            # GBP
    # GBPAUD
    elif data['gbp_low'].iloc[i] < -gbp_best_time and data['aud_high'].iloc[i] > gbp_best_time and flag_buy_gbpaud:
        if not flag_sell_gbpaud:
            balance_backtest = account_balance(balance_backtest,risk,leverage,price_sold_gbpaud,data['GBPAUD_Low'].iloc[i])
            list_backtest.append(balance_backtest)
            price_bought_gbpaud = data['GBPAUD_Low'].iloc[i]
            price_sold_gbpaud = 0
            flag_sell_gbpaud = True
            flag_buy_gbpaud = False
        else:
            price_bought_gbpaud = data['GBPAUD_Low'].iloc[i]
            flag_buy_gbpaud = False
    elif data['gbp_high'].iloc[i] > gbp_best_time and data['aud_low'].iloc[i] < -gbp_best_time and flag_sell_gbpaud:
        if not flag_buy_gbpaud:
            balance_backtest = account_balance(balance_backtest,risk,leverage,data['GBPAUD_High'].iloc[i],price_bought_gbpaud)
            list_backtest.append(balance_backtest)
            price_sold_gbpaud = data['GBPAUD_High'].iloc[i]
            price_bought_gbpaud = 0
            flag_buy_gbpaud = True
            flag_sell_gbpaud = False
        else:
            price_sold_gbpaud = data['GBPAUD_High'].iloc[i]
            flag_sell_gbpaud = False
    # GBPCHF
    elif data['gbp_low'].iloc[i] < -gbp_best_time and data['chf_high'].iloc[i] > gbp_best_time and flag_buy_gbpchf:
        if not flag_sell_gbpchf:
            balance_backtest = account_balance(balance_backtest,risk,leverage,price_sold_gbpchf,data['GBPCHF_Low'].iloc[i])
            list_backtest.append(balance_backtest)
            price_bought_gbpchf = data['GBPCHF_Low'].iloc[i]
            price_sold_gbpchf = 0
            flag_sell_gbpchf = True
            flag_buy_gbpchf = False
        else:
            price_bought_gbpchf = data['GBPCHF_Low'].iloc[i]
            flag_buy_gbpchf = False
    elif data['gbp_high'].iloc[i] > gbp_best_time and data['chf_low'].iloc[i] < -gbp_best_time and flag_sell_gbpchf:
        if not flag_buy_gbpchf:
            balance_backtest = account_balance(balance_backtest,risk,leverage,data['GBPCHF_High'].iloc[i],price_bought_gbpchf)
            list_backtest.append(balance_backtest)
            price_sold_gbpchf = data['GBPCHF_High'].iloc[i]
            price_bought_gbpchf = 0
            flag_buy_gbpchf = True
            flag_sell_gbpchf = False
        else:
            price_sold_gbpchf = data['GBPCHF_High'].iloc[i]
            flag_sell_gbpchf = False
    # GBPJPY
    elif data['gbp_low'].iloc[i] < -gbp_best_time and data['jpy_high'].iloc[i] > gbp_best_time and flag_buy_gbpjpy:
        if not flag_sell_gbpjpy:
            balance_backtest = account_balance(balance_backtest,risk,leverage,price_sold_gbpjpy,data['GBPJPY_Low'].iloc[i],jpy=True)
            list_backtest.append(balance_backtest)
            price_bought_gbpjpy = data['GBPJPY_Low'].iloc[i]
            price_sold_gbpjpy = 0
            flag_sell_gbpjpy = True
            flag_buy_gbpjpy = False
        else:
            price_bought_gbpjpy = data['GBPJPY_Low'].iloc[i]
            flag_buy_gbpjpy = False
    elif data['gbp_high'].iloc[i] > gbp_best_time and data['jpy_low'].iloc[i] < -gbp_best_time and flag_sell_gbpjpy:
        if not flag_buy_gbpjpy:
            balance_backtest = account_balance(balance_backtest,risk,leverage,data['GBPJPY_High'].iloc[i],price_bought_gbpjpy,jpy=True)
            list_backtest.append(balance_backtest)
            price_sold_gbpjpy = data['GBPJPY_High'].iloc[i]
            price_bought_gbpjpy = 0
            flag_buy_gbpjpy = True
            flag_sell_gbpjpy = False
        else:
            price_sold_gbpjpy = data['GBPJPY_High'].iloc[i]
            flag_sell_gbpjpy = False
    # GBPCAD
    elif data['gbp_low'].iloc[i] < -gbp_best_time and data['cad_high'].iloc[i] > gbp_best_time and flag_buy_gbpcad:
        if not flag_sell_gbpcad:
            balance_backtest = account_balance(balance_backtest,risk,leverage,price_sold_gbpcad,data['GBPCAD_Low'].iloc[i])
            list_backtest.append(balance_backtest)
            price_bought_gbpcad = data['GBPCAD_Low'].iloc[i]
            price_sold_gbpcad = 0
            flag_sell_gbpcad = True
            flag_buy_gbpcad = False
        else:
            price_bought_gbpcad = data['GBPCAD_Low'].iloc[i]
            flag_buy_gbpcad = False
    elif data['gbp_high'].iloc[i] > gbp_best_time and data['cad_low'].iloc[i] < -gbp_best_time and flag_sell_gbpcad:
        if not flag_buy_gbpcad:
            balance_backtest = account_balance(balance_backtest,risk,leverage,data['GBPCAD_High'].iloc[i],price_bought_gbpcad)
            list_backtest.append(balance_backtest)
            price_sold_gbpcad = data['GBPCAD_High'].iloc[i]
            price_bought_gbpcad = 0
            flag_buy_gbpcad = True
            flag_sell_gbpcad = False
        else:
            price_sold_gbpcad = data['GBPCAD_High'].iloc[i]
            flag_sell_gbpcad = False
    # GBPNZD
    elif data['gbp_low'].iloc[i] < -gbp_best_time and data['nzd_high'].iloc[i] > gbp_best_time and flag_buy_gbpnzd:
        if not flag_sell_gbpnzd:
            balance_backtest = account_balance(balance_backtest,risk,leverage,price_sold_gbpnzd,data['GBPNZD_Low'].iloc[i])
            list_backtest.append(balance_backtest)
            price_bought_gbpnzd = data['GBPNZD_Low'].iloc[i]
            price_sold_gbpnzd = 0
            flag_sell_gbpnzd = True
            flag_buy_gbpnzd = False
        else:
            price_bought_gbpnzd = data['GBPNZD_Low'].iloc[i]
            flag_buy_gbpnzd = False
    elif data['gbp_high'].iloc[i] > gbp_best_time and data['nzd_low'].iloc[i] < -gbp_best_time and flag_sell_gbpnzd:
        if not flag_buy_gbpnzd:
            balance_backtest = account_balance(balance_backtest,risk,leverage,data['GBPNZD_High'].iloc[i],price_bought_gbpnzd)
            list_backtest.append(balance_backtest)
            price_sold_gbpnzd = data['GBPNZD_High'].iloc[i]
            price_bought_gbpnzd = 0
            flag_buy_gbpnzd = True
            flag_sell_gbpnzd = False
        else:
            price_sold_gbpnzd = data['GBPNZD_High'].iloc[i]
            flag_sell_gbpnzd = False
            # JPY
    # AUDJPY
    elif data['aud_low'].iloc[i] < -jpy_best_time and data['jpy_high'].iloc[i] > jpy_best_time and flag_buy_audjpy:
        if not flag_sell_audjpy:
            balance_backtest = account_balance(balance_backtest,risk,leverage,price_sold_audjpy,data['AUDJPY_Low'].iloc[i],jpy=True)
            list_backtest.append(balance_backtest)
            price_bought_audjpy = data['AUDJPY_Low'].iloc[i]
            price_sold_audjpy = 0
            flag_sell_audjpy = True
            flag_buy_audjpy = False
        else:
            price_bought_audjpy = data['AUDJPY_Low'].iloc[i]
            flag_buy_audjpy = False
    elif data['aud_high'].iloc[i] > jpy_best_time and data['jpy_low'].iloc[i] < -jpy_best_time and flag_sell_audjpy:
        if not flag_buy_audjpy:
            balance_backtest = account_balance(balance_backtest,risk,leverage,data['AUDJPY_High'].iloc[i],price_bought_audjpy,jpy=True)
            list_backtest.append(balance_backtest)
            price_sold_audjpy = data['AUDJPY_High'].iloc[i]
            price_bought_audjpy = 0
            flag_buy_audjpy = True
            flag_sell_audjpy = False
        else:
            price_sold_audjpy = data['AUDJPY_High'].iloc[i]
            flag_sell_audjpy = False
    # CADJPY
    elif data['cad_low'].iloc[i] < -jpy_best_time and data['jpy_high'].iloc[i] > jpy_best_time and flag_buy_cadjpy:
        if not flag_sell_cadjpy:
            balance_backtest = account_balance(balance_backtest,risk,leverage,price_sold_cadjpy,data['CADJPY_Low'].iloc[i],jpy=True)
            list_backtest.append(balance_backtest)
            price_bought_cadjpy = data['CADJPY_Low'].iloc[i]
            price_sold_cadjpy = 0
            flag_sell_cadjpy = True
            flag_buy_cadjpy = False
        else:
            price_bought_cadjpy = data['CADJPY_Low'].iloc[i]
            flag_buy_cadjpy = False
    elif data['cad_high'].iloc[i] > jpy_best_time and data['jpy_low'].iloc[i] < -jpy_best_time and flag_sell_cadjpy:
        if not flag_buy_cadjpy:
            balance_backtest = account_balance(balance_backtest,risk,leverage,data['CADJPY_High'].iloc[i],price_bought_cadjpy,jpy=True)
            list_backtest.append(balance_backtest)
            price_sold_cadjpy = data['CADJPY_High'].iloc[i]
            price_bought_cadjpy = 0
            flag_buy_cadjpy = True
            flag_sell_cadjpy = False
        else:
            price_sold_cadjpy = data['CADJPY_High'].iloc[i]
            flag_sell_cadjpy = False
    # CHFJPY
    elif data['chf_low'].iloc[i] < -jpy_best_time and data['jpy_high'].iloc[i] > jpy_best_time and flag_buy_chfjpy:
        if not flag_sell_chfjpy:
            balance_backtest = account_balance(balance_backtest,risk,leverage,price_sold_chfjpy,data['CHFJPY_Low'].iloc[i],jpy=True)
            list_backtest.append(balance_backtest)
            price_bought_chfjpy = data['CHFJPY_Low'].iloc[i]
            price_sold_chfjpy = 0
            flag_sell_chfjpy = True
            flag_buy_chfjpy = False
        else:
            price_bought_chfjpy = data['CHFJPY_Low'].iloc[i]
            flag_buy_chfjpy = False
    elif data['chf_high'].iloc[i] > jpy_best_time and data['jpy_low'].iloc[i] < -jpy_best_time and flag_sell_chfjpy:
        if not flag_buy_chfjpy:
            balance_backtest = account_balance(balance_backtest,risk,leverage,data['CHFJPY_High'].iloc[i],price_bought_chfjpy,jpy=True)
            list_backtest.append(balance_backtest)
            price_sold_chfjpy = data['CHFJPY_High'].iloc[i]
            price_bought_chfjpy = 0
            flag_buy_chfjpy = True
            flag_sell_chfjpy = False
            price_sold_chfjpy = data['CHFJPY_High'].iloc[i]
        else:
            flag_sell_chfjpy = False
    # NZDJPY
    elif data['nzd_low'].iloc[i] < -jpy_best_time and data['jpy_high'].iloc[i] > jpy_best_time and flag_buy_nzdjpy:
        if not flag_sell_nzdjpy:
            balance_backtest = account_balance(balance_backtest,risk,leverage,price_sold_nzdjpy,data['NZDJPY_Low'].iloc[i],jpy=True)
            list_backtest.append(balance_backtest)
            price_bought_nzdjpy = data['NZDJPY_Low'].iloc[i]
            price_sold_nzdjpy = 0
            flag_sell_nzdjpy = True
            flag_buy_nzdjpy = False
        else:
            price_bought_nzdjpy = data['NZDJPY_Low'].iloc[i]
            flag_buy_nzdjpy = False
    elif data['nzd_high'].iloc[i] > jpy_best_time and data['jpy_low'].iloc[i] < -jpy_best_time and flag_sell_nzdjpy:
        if not flag_buy_nzdjpy:
            balance_backtest = account_balance(balance_backtest,risk,leverage,data['NZDJPY_High'].iloc[i],price_bought_nzdjpy,jpy=True)
            list_backtest.append(balance_backtest)
            price_sold_nzdjpy = data['NZDJPY_High'].iloc[i]
            price_bought_nzdjpy = 0
            flag_buy_nzdjpy = True
            flag_sell_nzdjpy = False
        else:
            price_sold_nzdjpy = data['NZDJPY_High'].iloc[i]
            flag_sell_nzdjpy = False
            # CHF
    # AUDCHF
    elif data['aud_low'].iloc[i] < -chf_besttime and data['chf_high'].iloc[i] > chf_besttime and flag_buy_audchf:
        if not flag_sell_audchf:
            balance_backtest = account_balance(balance_backtest,risk,leverage,price_sold_audchf,data['AUDCHF_Low'].iloc[i])
            list_backtest.append(balance_backtest)
            price_bought_audchf = data['AUDCHF_Low'].iloc[i]
            price_sold_audchf = 0
            flag_sell_audchf = True
            flag_buy_audchf = False
        else:
            price_bought_audchf = data['AUDCHF_Low'].iloc[i]
            flag_buy_audchf = False
    elif data['aud_high'].iloc[i] > chf_besttime and data['chf_low'].iloc[i] < -chf_besttime and flag_sell_audchf:
        if not flag_buy_audchf:
            balance_backtest = account_balance(balance_backtest,risk,leverage,data['AUDCHF_High'].iloc[i],price_bought_audchf)
            list_backtest.append(balance_backtest)
            price_sold_audchf = data['AUDCHF_High'].iloc[i]
            price_bought_audchf = 0
            flag_buy_audchf = True
            flag_sell_audchf = False
        else:
            price_sold_audchf = data['AUDCHF_High'].iloc[i]
            flag_sell_audchf = False
    # CADCHF
    elif data['cad_low'].iloc[i] < -chf_besttime and data['chf_high'].iloc[i] > chf_besttime and flag_buy_cadchf:
        if not flag_sell_cadchf:
            balance_backtest = account_balance(balance_backtest,risk,leverage,price_sold_cadchf,data['CADCHF_Low'].iloc[i])
            list_backtest.append(balance_backtest)
            price_bought_cadchf = data['CADCHF_Low'].iloc[i]
            price_sold_cadchf = 0
            flag_sell_cadchf = True
            flag_buy_cadchf = False
        else:
            price_bought_cadchf = data['CADCHF_Low'].iloc[i]
            flag_buy_cadchf = False
    elif data['cad_high'].iloc[i] > chf_besttime and data['chf_low'].iloc[i] < -chf_besttime and flag_sell_cadchf:
        if not flag_buy_audchf:
            balance_backtest = account_balance(balance_backtest,risk,leverage,data['CADCHF_High'].iloc[i],price_bought_cadchf)
            list_backtest.append(balance_backtest)
            price_sold_cadchf = data['CADCHF_High'].iloc[i]
            price_bought_cadchf = 0
            flag_buy_cadchf = True
            flag_sell_cadchf = False
        else:
            price_sold_cadchf = data['CADCHF_High'].iloc[i]
            flag_sell_cadchf = False
    # NZDCHF
    elif data['nzd_low'].iloc[i] < -chf_besttime and data['chf_high'].iloc[i] > chf_besttime and flag_buy_nzdchf:
        if not flag_sell_nzdchf:
            balance_backtest = account_balance(balance_backtest,risk,leverage,price_sold_nzdchf,data['NZDCHF_Low'].iloc[i])
            list_backtest.append(balance_backtest)
            price_bought_nzdchf = data['NZDCHF_Low'].iloc[i]
            price_sold_nzdchf = 0
            flag_sell_nzdchf = True
            flag_buy_nzdchf = False
        else:
            price_bought_nzdchf = data['NZDCHF_Low'].iloc[i]
            flag_buy_nzdchf = False
    elif data['nzd_high'].iloc[i] > chf_besttime and data['chf_low'].iloc[i] < -chf_besttime and flag_sell_nzdchf:
        if not flag_buy_nzdchf:
            balance_backtest = account_balance(balance_backtest,risk,leverage,data['NZDCHF_High'].iloc[i],price_bought_nzdchf)
            list_backtest.append(balance_backtest)
            price_sold_nzdchf = data['NZDCHF_High'].iloc[i]
            price_bought_nzdchf = 0
            flag_buy_nzdchf = True
            flag_sell_nzdchf = False
        else:
            price_sold_nzdchf = data['NZDCHF_High'].iloc[i]
            flag_sell_nzdchf = False
            # NZD
    # AUDNZD
    elif data['aud_low'].iloc[i] < -nzd_best_time and data['nzd_high'].iloc[i] > nzd_best_time and flag_buy_audnzd:
        if not flag_sell_audnzd:
            balance_backtest = account_balance(balance_backtest,risk,leverage,price_sold_audnzd,data['AUDNZD_Low'].iloc[i])
            list_backtest.append(balance_backtest)
            price_bought_audnzd = data['AUDNZD_Low'].iloc[i]
            price_sold_audnzd = 0
            flag_sell_audnzd = True
            flag_buy_audnzd = False
        else:
            price_bought_audnzd = data['AUDNZD_Low'].iloc[i]
            flag_buy_audnzd = False
    elif data['aud_high'].iloc[i] > nzd_best_time and data['nzd_low'].iloc[i] < -nzd_best_time and flag_sell_audnzd:
        if not flag_buy_audnzd:
            balance_backtest = account_balance(balance_backtest,risk,leverage,data['AUDNZD_High'].iloc[i],price_bought_audnzd)
            list_backtest.append(balance_backtest)
            price_sold_audnzd = data['AUDNZD_High'].iloc[i]
            price_bought_audnzd = 0
            flag_buy_audnzd = True
            flag_sell_audnzd = False
        else:
            price_sold_audnzd = data['AUDNZD_High'].iloc[i]
            flag_sell_audnzd= False
    # NZDCAD
    elif data['nzd_low'].iloc[i] < -nzd_best_time and data['cad_high'].iloc[i] > nzd_best_time and flag_buy_nzdcad:
        if not flag_sell_nzdcad:
            balance_backtest = account_balance(balance_backtest,risk,leverage,price_sold_nzdcad,data['NZDCAD_Low'].iloc[i])
            list_backtest.append(balance_backtest)
            price_bought_nzdcad = data['NZDCAD_Low'].iloc[i]
            price_sold_nzdcad = 0
            flag_sell_nzdcad = True
            flag_buy_nzdcad = False
        else:
            price_bought_nzdcad = data['NZDCAD_Low'].iloc[i]
            flag_buy_nzdcad = False
    elif data['nzd_high'].iloc[i] > nzd_best_time and data['cad_low'].iloc[i] < -nzd_best_time and flag_sell_nzdcad:
        if not flag_buy_nzdcad:
            balance_backtest = account_balance(balance_backtest,risk,leverage,data['NZDCAD_High'].iloc[i],price_bought_nzdcad)
            list_backtest.append(balance_backtest)
            price_sold_nzdcad = data['NZDCAD_High'].iloc[i]
            price_bought_nzdcad = 0
            flag_buy_nzdcad = True
            flag_sell_nzdcad = False
        else:
            price_sold_nzdcad = data['NZDCAD_High'].iloc[i]
            flag_sell_nzdcad = False
            # AUD
    # AUDCAD
    elif data['aud_low'].iloc[i] < -aud_best_time and data['cad_high'].iloc[i] > aud_best_time and flag_buy_audcad:
        if not flag_sell_audcad:
            balance_backtest = account_balance(balance_backtest,risk,leverage,price_sold_audcad,data['AUDCAD_Low'].iloc[i])
            list_backtest.append(balance_backtest)
            price_bought_audcad = data['AUDCAD_Low'].iloc[i]
            price_sold_audcad = 0
            flag_sell_audcad = True
            flag_buy_audcad = False
        else:
            price_bought_audcad = data['AUDCAD_Low'].iloc[i]
            flag_buy_audcad = False
    elif data['aud_high'].iloc[i] > aud_best_time and data['cad_low'].iloc[i] < -aud_best_time and flag_sell_audcad:
        if not flag_buy_audcad:
            balance_backtest = account_balance(balance_backtest,risk,leverage,data['AUDCAD_High'].iloc[i],price_bought_audcad)
            list_backtest.append(balance_backtest)
            price_sold_audcad = data['AUDCAD_High'].iloc[i]
            price_bought_audcad = 0
            flag_buy_audcad = True
            flag_sell_audcad = False
        else:
            price_sold_audcad = data['AUDCAD_High'].iloc[i]
            flag_sell_audcad = False
